In [1]:
! pip install httpx

In [2]:
import json
import httpx

In [7]:
client = httpx.Client(
    headers={
        # this is internal ID of an instegram backend app. It doesn't change often.
        "x-ig-app-id": "936619743392459",
        # use browser-like features
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9,ru;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept": "*/*",
    }
)


def scrape_user(username: str):
    """Scrape Instagram user's data"""
    result = client.get(
        f"https://i.instagram.com/api/v1/users/web_profile_info/?username={username}",
    )
    data = json.loads(result.content)
    return data["data"]["user"]

test = scrape_user("mattrife")
# readable json in ipynb
print(json.dumps(test, indent=4))

{
    "ai_agent_type": null,
    "biography": "NEW SPECIAL ON NETFLIX! #Lucid\nStandup Comedian/Actor/Person. \u2764\ufe0f TOUR DATES \ud83d\udc47\ud83c\udffc",
    "bio_links": [
        {
            "title": "",
            "lynx_url": "https://l.instagram.com/?u=http%3A%2F%2Fmattrifeofficial.com%2F&e=AT06KMSvzxMQBH5XcLDRBoYCOzqrAXS668RV5DK-57YlLsv8DSgDRupdaDvd9oVLQilIpOGSa6voaeUi9yLWU5UkzSJyWtSo",
            "url": "http://mattrifeofficial.com",
            "link_type": "external"
        }
    ],
    "fb_profile_biolink": null,
    "biography_with_entities": {
        "raw_text": "NEW SPECIAL ON NETFLIX! #Lucid\nStandup Comedian/Actor/Person. \u2764\ufe0f TOUR DATES \ud83d\udc47\ud83c\udffc",
        "entities": [
            {
                "user": null,
                "hashtag": {
                    "name": "lucid"
                }
            }
        ]
    },
    "blocked_by_viewer": false,
    "restricted_by_viewer": null,
    "country_block": false,
    "eimu_id": "11

In [8]:
! pip install jmespath

In [9]:
import jmespath

def parse_user(data: dict) -> dict:
    """Parse instagram user's hidden web dataset for user's data"""
    print("parsing user data {}", data['username'])
    result = jmespath.search(
        """{
        name: full_name,
        username: username,
        id: id,
        category: category_name,
        business_category: business_category_name,
        image_count: edge_owner_to_timeline_media.count,
        images: edge_felix_video_timeline.edges[].node.{
            id: id, 
            title: title,
            shortcode: shortcode,
            src: display_url,
            url: video_url,
            captions: edge_media_to_caption.edges[].node.text
        }
    }""",
        data,
    )
    return result

In [10]:
google_test_parsed = parse_user(test)

# readable json in ipynb
print(json.dumps(google_test_parsed, indent=4))

parsing user data {} mattrife
{
    "name": "Matt Rife",
    "username": "mattrife",
    "id": "54284618",
    "category": "Comedian",
    "business_category": null,
    "image_count": 1532,
    "images": [
        {
            "id": "2887535143857045365",
            "title": "",
            "shortcode": "CgSleoeDj91",
            "src": "https://scontent-lax3-2.cdninstagram.com/v/t51.2885-15/294801159_1685641258477595_117712680669166960_n.jpg?stp=dst-jpg_e35_s1080x1080_sh0.08_tt6&_nc_ht=scontent-lax3-2.cdninstagram.com&_nc_cat=111&_nc_oc=Q6cZ2AHi85oqueYgngIUh-WFT2Cwl4fwee6onsVnuRIvrTBmq5MUk91U_YP33-DdRqxpJAE&_nc_ohc=0N_DKj08rsIQ7kNvgGpsF2j&_nc_gid=227fb7a5218942978050e5f64a697639&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AYHmQZRzjsClZVc5VqLGLkgQDARJ4QTf1ZHVTSCl7sqMHQ&oe=67D2AF86&_nc_sid=8b3546",
            "url": "https://scontent-lax3-2.cdninstagram.com/o1/v/t16/f2/m69/AQNORbm8dx7C-fonW4j3aQ5WYEOJ3s_TvKaqqCUKjDCES1iql5xq8z4qfrJxgVYv2hCbjsPefseFrVSE4ASbDrKI.mp4?stp=dst-mp4&efg=eyJxZV9ncm91cHM

In [11]:
! pip install pandas
import pandas as pd

In [12]:
print(len(google_test_parsed['images']))

12


In [13]:
google_test_parsed['images'][0]['id']

google_test_parsed['images'][0]['shortcode']



data = [(i['id'], i['shortcode'],i['url'],i['captions']) for i in google_test_parsed['images']]

#put this in a pandas dataframe

df = pd.DataFrame(data, columns=['id', 'shortcode', 'url', 'captions'])
df.head()


,id,shortcode,url,captions
0,2887535143857045365,CgSleoeDj91,https://scontent-lax3-2.cdninstagram.com/o1/v/...,[Don’t just “be alive”…. LIVE. 🪂 My friends an...
1,2791163856868579404,Ca8NL1Wj5hM,https://scontent-lax3-1.cdninstagram.com/o1/v/...,[Thank you @papermagazine for exposing my secr...
2,2783986728239983869,CaitS3cJWz9,https://scontent-lax3-2.cdninstagram.com/o1/v/...,[Full Version of this man’s wife throwing him ...
3,2764980359854330363,CZfLvzgDJH7,https://scontent-lax3-1.cdninstagram.com/o1/v/...,[#NorthOfThe10 premieres Feb. 10th on @betplus...
4,2763639334808381326,CZaa1UmjOeO,https://scontent-lax3-2.cdninstagram.com/o1/v/...,[FEB. 10th!!! The premiere of our new film #No...


In [14]:
import httpx
import json
from typing import Dict
from urllib.parse import quote


In [15]:
def scrape_post(ig_doc_id ,url_or_shortcode: str) -> Dict:
    """Scrape single Instagram post data"""
    print(f"Scraping: {url_or_shortcode}, id: {ig_doc_id}")
    
    if "http" in url_or_shortcode:
        shortcode = url_or_shortcode.split("/p/")[-1].split("/")[0]
    else:
        shortcode = url_or_shortcode

    variables = quote(json.dumps({
        'shortcode':shortcode,'fetch_tagged_user_count':None,
        'hoisted_comment_id':None,'hoisted_reply_id':None
    }, separators=(',', ':')))
    body = f"variables={variables}&doc_id={ig_doc_id}"
    url = "https://www.instagram.com/graphql/query"

    result = httpx.post(
        url=url,
        headers={"content-type": "application/x-www-form-urlencoded"},
        data=body
    )
    data = json.loads(result.content)
    
    print(data)
    
    # return data["data"]["xdt_shortcode_media"]
    return data

In [ ]:
# document id for content query:
#more post content: 
# IG_DOC_ID = "9195084877213487"
#more content
# IG_DOC_ID = "9507655305965624"



#relationships/related : 9480525715305126
# IG_DOC_ID = "9480525715305126"
#acc metadata: 9109150515847101
# IG_DOC_ID = "9109150515847101"
#reels: 9318096484942317
# IG_DOC_ID = "9318096484942317"

Scraping: CgkkVI8jGwA, id: 9109150515847101
{'errors': [{'message': 'execution error', 'severity': 'CRITICAL'}], 'data': None, 'status': 'ok'}


In [60]:
print(posts)

{'data': {'xdt_shortcode_media': None}, 'extensions': {'is_final': True}, 'status': 'ok'}


In [ ]:
post_url = "https://www.instagram.com/p/"



In [16]:
import requests
import re
import json

def post_scrape(shortcode: str, mediaid: str):
    session = requests.Session()

    # Step 1: Fetch the webpage HTML to extract the media ID
    url = f"https://www.instagram.com/p/{shortcode}/"
    response : requests.Response = session.get(url)
    html_content:str = response.text
    print(f"Initial request: {response.status_code}")

    # Step 2: Use media ID to fetch JSON data
    api_url = f"https://instagram.com/api/v1/media/{mediaid}/info/"

    headers = {
        "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Mobile Safari/537.36",
        "Accept": "application/json",
        "Accept-Language": "en-US,en;q=0.9",
    }

    api_response = session.get(api_url, headers=headers)

    if api_response.status_code != 200:
        raise ConnectionError(f"API request failed with status code {api_response.status_code}")

    return api_response.json()

# Example usage:
post_data = post_scrape(df[0:1]['shortcode'][0], df[0:1]['id'][0])
print(json.dumps(post_data, indent=2))

Initial request: 200


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [104]:
df[0:1]['id'][0]

'2887535143857045365'

In [17]:
 ! pip install pytest-playwright
 ! playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 16.2 MB/s eta 0:00:0000:0100:01
123.3 MiB [                    ] 0% 0.0s123.3 MiB [                    ] 0% 7.7s123.3 MiB [                    ] 0% 6.4s123.3 MiB [                    ] 0% 5.5s123.3 MiB [                    ] 1% 6.7s123.3 MiB [                    ] 1% 7.1s123.3 MiB [                    ] 2% 4.8s123.3 MiB [                    ] 2% 8.9s123.3 MiB [                    ] 2% 8.3s123.3 MiB [=                   ] 2% 7.7s123.3 MiB [=                   ] 3% 7.4s123.3 MiB [=                   ] 3% 7.0s123.3 MiB [=                   ] 4% 6.7s123.3 MiB [=                   ] 4% 6.4s123.3 MiB [=                   ] 4% 6.2s123.3 MiB [=                   ] 5% 6.1s123.3 MiB [=                   ] 5% 5.9s123.3 MiB [=                   ] 6% 5.7s123.3 MiB [=                   ] 6% 5.6s123.3 MiB [=                   ] 7% 5.4s123.3 MiB [=                   ] 7% 5.3s123.3 MiB [==                  ] 7% 5.2s123.3 MiB [==                  

In [18]:
import asyncio
from playwright.async_api import async_playwright
import re
import json

In [24]:
# Global references for easy access across notebook cells
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)  # Browser will stay open
context = await browser.new_context(
    user_agent="Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) "
               "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Mobile Safari/537.36"
)
page = await context.new_page()
# You now have persistent global references:
# playwright, browser, context, page

In [26]:



async def scrape_instagram_post(shortcode: str):
    target_endpoint_pattern = r"/api/v1/media/\d+/info/"

    post_json = {}

    async with async_playwright() as p:
        

        # Intercept network responses
        page.on("response", lambda response: asyncio.ensure_future(
            intercept_response(response, target_endpoint_pattern, post_json)
        ))

        # Navigate to Instagram post page (this triggers the desired API request)
        post_url = f"https://www.instagram.com/p/{shortcode}/"
        await page.goto(post_url, wait_until="networkidle")


        if not post_json:
            raise ValueError("Failed to intercept the desired JSON response.")

        return post_json

async def intercept_response(response, pattern, post_json):
    if re.search(pattern, response.url):
        if response.status == 200:
            print(f"Intercepted URL: {response.url}")
            data = await response.json()
            post_json.update(data)

shortcode = 'DG6uwZKACNm'  # Replace with your actual shortcode
post_data = await scrape_instagram_post(shortcode)
print(json.dumps(post_data, indent=2))


TimeoutError: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://www.instagram.com/p/DG6uwZKACNm/", waiting until "networkidle"
